<a href="https://colab.research.google.com/github/JesseTNRoberts/Do-Large-Language-Models-Learn-Human-Like-Strategic-Preferences/blob/main/PopulationLM_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import gc

from google.colab import drive
drive.mount('/content/drive')

!pip install git+https://github.com/JesseTNRoberts/minicons_modded
!pip install git+https://github.com/JesseTNRoberts/PopulationLM
!pip install accelerate transformers

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
  Cloning https://github.com/JesseTNRoberts/minicons_modded to /tmp/pip-req-build-m3wfze7m
  Running command git clone --filter=blob:none --quiet https://github.com/JesseTNRoberts/minicons_modded /tmp/pip-req-build-m3wfze7m
  Resolved https://github.com/JesseTNRoberts/minicons_modded to commit 2aa9e4e05fb5c2af99f10dc75cc1e2968b5eaceb
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/JesseTNRoberts/PopulationLM to /tmp/pip-req-build-hkm3yvvt
  Running command git clone --filter=blob:none --quiet https://github.com/JesseTNRoberts/PopulationLM /tmp/pip-req-build-hkm3yvvt
  Resolved https://github.com/JesseTNRoberts/PopulationLM to commit e7adde2326e644d7c94d2c7086d9561f8aefb977
  Preparing metadata (setup.py) ... done
  Using cached accelerate-0.28.0-py3-none-any.whl (290 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_

In [ ]:

import os
import argparse
import csv
from tqdm import tqdm
import numpy as np
import random
import accelerate

import transformers
import os
import shutil
import glob

import torch
from transformers import AutoModelForMaskedLM, AutoModelForCausalLM
from transformers import AutoModelForMaskedLM, AutoModelForCausalLM
from torch.utils.data import DataLoader

from minicons import scorer
import PopulationLM as pop

from google.colab import userdata
token = userdata.get('HF_TOKEN')


committee_size = 50
add_dropout_layers = True


inpath = '/content/drive/MyDrive/Data/TravelersDilemmaBinaryHS.csv'
# inpath = '/content/drive/MyDrive/Data/rational.csv'
zippath = '/content/drive/MyDrive/Data/cache/'
cachepath = '/content/cache/'



# make results dir: ../data/typicality/results/(dataset)/model_name.csv
components = inpath.split("/")
data_dir = "/".join(components[0:-1])
dataset_name = components[-1].split(".")[0]
results_dir = f"{data_dir}/results/rational"

dataset = []
with open(inpath, "r") as f:
    reader = csv.DictReader(f)
    column_names = reader.fieldnames
    for row in reader:
        dataset.append(list(row.values()))

batch_size = 108
device = 'cuda'
stimuli_loader = DataLoader(dataset, batch_size = batch_size, num_workers=0)




models = [
    ['bert-base-uncased',  'mlm'],
    ['bert-large-uncased', 'mlm'],
    ['distilbert-base-uncased', 'mlm'],
    ['roberta-base', 'mlm'],
    ['roberta-large', 'mlm'],
    ["google/electra-large-generator", 'mlm'],
    ['facebook/xlm-roberta-xl', 'mlm'],
    ['distilgpt2', 'incremental'],
    ['gpt2-medium',  'incremental'],
    ['gpt2',  'incremental'],
    ['openai-gpt',  'incremental'],
    ["daryl149/llama-2-7b-hf", 'incremental'],
    ["mistralai/Mistral-7B-v0.1",  'incremental'],
    ["google/gemma-7b",  'incremental'],
    ['PKU-Alignment/alpaca-7b-reproduced', 'incremental'],
    ['openlm-research/open_llama_13b',  'incremental'],
    ['upstage/SOLAR-10.7B-v1.0',  'incremental'],
    ['microsoft/phi-2',  'incremental'],
    ['meta-llama/Llama-2-13b-hf',  'incremental'],
]

use_population = True
local_only = False

In [ ]:


for model in models:
  # don't repeat completed experiments
  model_name = model[0].replace("/", "_")
  if os.path.exists(results_dir + f"/{model_name}.csv"):
    continue

  print(model[0])

  if model[1] == "mlm":
    try:
      transformer = scorer.MaskedLMScorer(model[0], device, token=token, cache_dir=cachepath,
                                          local_files_only=local_only, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto")
    except:
      transformer = scorer.MaskedLMScorer(model[0], device, token=token, cache_dir=cachepath, local_files_only=local_only)

  elif model[1] == "incremental":
    # Try to load the model with fast parameters
    try:
      transformer = scorer.IncrementalLMScorer(model[0], device, token=token, cache_dir=cachepath,
                                          local_files_only=local_only, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto")

    # Sometimes the above fails because old models don't implement the necessary functions
    # call without fast parameters
    except:
      transformer = scorer.IncrementalLMScorer(model[0], device, token=token, cache_dir=cachepath, local_files_only=local_only)

  else:
    raise ValueError("Invalid model type")

  num_params = [sum(p.numel() for p in transformer.model.parameters())] * len(dataset)

  for val in [0.1]:

    # convert the internal model to use MC Dropout with populationLM
    if use_population:
      pop.DropoutUtils.convert_dropouts(transformer.model)
      pop.DropoutUtils.activate_mc_dropout(transformer.model, activate=True, random=val)

    results = []
    control_results = []
    conclusion_only = []

    # create a lambda function alias for the method that performs generates the probabilities
    call_me = lambda prefixes, queries: transformer.conditional_score(prefixes, queries, reduction=lambda x: (x.sum(0).item(), x.mean(0).item(), x.tolist()))

    for batch in stimuli_loader:
        premise = list(batch[0])
        conclusion = list(batch[1])

        # this line generates the dropout population
        if use_population:
          population = pop.generate_dropout_population(transformer.model, lambda: call_me(premise, conclusion), committee_size=committee_size)

        print(len(premise), len(conclusion))

        # this line gets the probabilities for the dropout population
        if use_population:
          outs = [item for item in tqdm(pop.call_function_with_population(transformer.model, population, lambda: call_me(premise, conclusion)),
                                        total=committee_size)]
        else:
          outs = [call_me(premise, conclusion)]

        transposed_outs = [[row[i] for row in outs] for i in range(len(outs[0]))]
        priming_scores = [score for score in transposed_outs]
        results.extend(priming_scores)

    data_out = list(zip(*dataset))
    new_col_names = column_names

    data_out.append(results)
    new_col_names += ["score (sum, mean, [list)"]

    data_out.append(num_params)
    data_out.append([model_name] * len(results))
    new_col_names += ["params", "model"]

    with open(results_dir + f"/{model_name}"+".csv", "w") as f:
        writer = csv.writer(f)
        writer.writerow(new_col_names)
        writer.writerows(list(zip(*data_out)))
        f.flush()


mistralai/Mistral-7B-v0.1


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

trying to add dropout layers...
[MistralModel(
  (embed_tokens): Embedding(32000, 4096)
  (layers): ModuleList(
    (0-31): 32 x MistralDecoderLayer(
      (self_attn): MistralSdpaAttention(
        (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
        (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        (rotary_emb): MistralRotaryEmbedding()
      )
      (mlp): MistralMLP(
        (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
        (down_proj): Sequential(
          (0): Linear(in_features=14336, out_features=4096, bias=False)
          (1): StratifiedDropoutMC()
        )
        (act_fn): SiLU()
      )
      (input_layernorm): MistralRMSNorm()
      (post_attention_layernorm): MistralRMSNorm()
 

100%|██████████| 50/50 [00:07<00:00,  6.44it/s]


google/gemma-7b


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

trying to add dropout layers...
[GemmaModel(
  (embed_tokens): Embedding(256000, 3072, padding_idx=0)
  (layers): ModuleList(
    (0-27): 28 x GemmaDecoderLayer(
      (self_attn): GemmaSdpaAttention(
        (q_proj): Linear(in_features=3072, out_features=4096, bias=False)
        (k_proj): Linear(in_features=3072, out_features=4096, bias=False)
        (v_proj): Linear(in_features=3072, out_features=4096, bias=False)
        (o_proj): Linear(in_features=4096, out_features=3072, bias=False)
        (rotary_emb): GemmaRotaryEmbedding()
      )
      (mlp): GemmaMLP(
        (gate_proj): Linear(in_features=3072, out_features=24576, bias=False)
        (up_proj): Linear(in_features=3072, out_features=24576, bias=False)
        (down_proj): Sequential(
          (0): Linear(in_features=24576, out_features=3072, bias=False)
          (1): StratifiedDropoutMC()
        )
        (act_fn): GELUActivation()
      )
      (input_layernorm): GemmaRMSNorm()
      (post_attention_layernorm): Gemm

100%|██████████| 50/50 [00:09<00:00,  5.55it/s]
